# Make SCA and network stacks

In [1]:
# Imports
import os
import glob
import numpy as np
import pandas as pd
import nibabel as nib
import multiprocessing as mp

In [2]:
# Paths
in_data = '/data1/guilimin/abide/abide_release_gsc0_lp01/'
abide_ref_path = '/data1/abide/Pheno/psm/abide_qc_maybe.csv'
in_pheno = '/data1/abide/Pheno/psm/abide_full_maybe.csv'
out_data = '/data1/guilimin/abide/sca/sc12'
net_mask = '/data1/cambridge/template/template_cambridge_basc_multiscale_sym_scale012.nii.gz'
brain_mask = '/data1/cambridge/template/template_mask.nii.gz'
f_tmp = '{}/fmri_{:07}_session_{}_run{}.nii.gz'

In [3]:
# Get the mask
n_img = nib.load(net_mask)
net = n_img.get_data()
m_img = nib.load(brain_mask)
mask = m_img.get_data().astype(bool)

In [4]:
# Get the pheno file
ref = pd.read_csv(abide_ref_path)
pheno = pd.read_csv(in_pheno)
pheno = pheno[['SUB_ID', 'distance', 'weights']]
merged = pd.merge(ref, pheno, on='SUB_ID', how='inner')

In [5]:
sites = ['Pitt', 'Olin', 'OHSU', 'SDSU', 'Trinity', 'UM_1', 'UM_2', 'USM', 'Yale', 'CMU_a', 'CMU_b',
        'Leuven_1', 'Leuven_2', 'KKI', 'NYU', 'Stanford', 'UCLA_1', 'UCLA_2',
         'Max_Mun', 'Caltech', 'SBL']

In [6]:
u_sites = list(merged.SITE_ID.unique())
l_sites = [x.lower() for x in u_sites]
# Get the index matches between all sites and these sites
s_ind = [(si, l_sites.index(site.lower())) if site.lower() in l_sites else (np.nan, np.nan) for si, site in enumerate(sites)]

In [7]:
# Make a dictionary
name_dict = dict()
for s in s_ind:
    if not any(np.isnan(s)):
        i_all, i_loc = s
        name_dict[u_sites[i_loc]] = sites[i_all]

In [8]:
# Use the name dict to replace the site names
merged.SITE_ID.replace(name_dict, inplace=True)

In [9]:
# Make some paths
file_paths = [os.path.join(in_data,f_tmp.format(row.SITE_ID, row.SUB_ID, row.session, row.run)) for index, row in merged.iterrows()]

In [10]:
# Test the paths
for fp in file_paths:
    if not os.path.isfile(fp) and not os.path.islink(fp):
        print('Something wrong with {}'.format(fp))

Looks like we got everything, good. Let's make some maps

In [4]:
0.5*np.log((1+0.9)/(1-0.9))

1.4722194895832204

In [5]:
import sys

In [10]:
a = '/data1/guilimin/abide/script'
b = 'abide/pheno/abide_qc_maybe.csv'

In [11]:
os.path.join(a,b)

'/data1/guilimin/abide/script/abide/pheno/abide_qc_maybe.csv'

In [ ]:
if not os.path.isdir()

In [ ]:
os.makedirs

In [7]:
sys.argv[0]

'/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py'

In [ ]:
def run_stuff(args):
    sub_path, net_mask, brain_mask, out_data = args
    
    # Check if anything should be done
    out_path = os.path.join(out_data, 'netstack_{}'.format(os.path.basename(sub_path)))
    if os.path.isfile(out_path):
        return 'Exists already: {}'.format(out_path)
    
    # Get the mask
    n_img = nib.load(net_mask)
    net = n_img.get_data()
    m_img = nib.load(brain_mask)
    mask = m_img.get_data().astype(bool)

    # Get the subject
    f_img = nib.load(sub_path)
    vol = f_img.get_data()
    # Vectorize it
    vec = vol[mask, ...]
    
    # Vectorize it
    vec_vol = vol[mask, ...]
    vec_net = net[mask]
    # Get the number of voxels
    n_vox = np.sum(mask)
    nets = np.unique(net[net!=0])
    n_net = len(nets)
    
    seed_maps = np.zeros((n_vox, n_net))
    for n_id, n_val in enumerate(nets):
        # Get the average signal in the ROI
        net_mean = np.mean(vec_vol[vec_net==n_val],0)
        # Compute the correlation vector
        corr_vec = np.array([0.5*np.log((1+np.corrcoef(i, net_mean)[1][0])/(1-np.corrcoef(i, net_mean)[1][0])) for i in vec_vol])
        seed_maps[:, n_id] = corr_vec

    # Generate the network stack
    template = np.zeros((mask.shape + (n_net,)))
    template[mask, ...] = seed_maps
    img = nib.Nifti1Image(template, header=n_img.get_header(), affine=n_img.get_affine())
    # Save the template
    nib.save(img, out_path)
    
    return 'Created new: {}'.format(out_path)

In [ ]:
# Set up some workers
args = [(fp, net_mask, brain_mask, out_data) for fp in file_paths]
pool = mp.Pool(3)
m = pool.map(run_stuff, args)

In [ ]:
for i in m:
    print(i)